# Utiliser des LLM en API pour générer avec une prompt (zero shot)

- des annotations : quels sont les articles qui mentionnent des thématiques IA
- de l'extraction d'information : quelles sont les notions utilisées

Quelques lectures 

- Stuhler, O., Ton, C. D., & Ollion, E. (2025). From Codebooks to Promptbooks: Extracting Information from Text with Generative Large Language Models. Sociological Methods & Research, 54(3), 794-848. https://doi.org/10.1177/00491241251336794 (Original work published 2025)
- https://www.css.cnrs.fr/classification-with-generative-llms-and-api-calls/

## Quelques mots sur le génératif

- Prédiction du next token
- Modèles pré-entrainés
    - des tailles & alignements différents
- Dépendances matérielles fortes

## Quels modèles ?

- vaste question
- équilibre puissance/coût (financier et matériel)
- enjeu de la sécurité des données

De nombreux prestataires

- openai, etc.
- openrouter
- humanum...

## Charger les données

In [18]:
import pandas as pd
df = pd.read_excel("../data/data.xlsx")
df = df[df["abstract"].notna()]
df.head()

,Unnamed: 0,id,title,publication_year,abstract
0,0,https://openalex.org/W2159397589,Computational Social Science,2009.0,A field is emerging that leverages the capacit...
2,2,https://openalex.org/W3081158114,Computational social science: Obstacles and op...,2020.0,"Data sharing, research ethics, and incentives ..."
3,3,https://openalex.org/W3022499311,Computational Social Science and Sociology,2020.0,The integration of social science with compute...
5,5,https://openalex.org/W4389636360,Can Large Language Models Transform Computatio...,2023.0,Abstract Large language models (LLMs) are capa...
7,7,https://openalex.org/W2790659496,Computational social science,2010.0,Abstract The social sciences investigate human...


## Charger une clé

In [3]:
import yaml
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)
api_key = config["api_key"]

## Utiliser openrouter et l'API openapi

Quickstart : https://platform.openai.com/docs/quickstart?api-mode=chat&language=python

In [6]:
#!pip install openai
from openai import OpenAI
api_url = "https://openrouter.ai/api/v1"
 
client = OpenAI(
  base_url=api_url,
  api_key=api_key,
)

response = client.chat.completions.create(
    model="openai/gpt-oss-120b",
    messages=[{"role": "user", "content": "Que peux tu dire de l'école Centrale Marseille ?"}]
)
response

ChatCompletion(id='gen-1771589879-EoJMnC4Sdzsa2AMWamVm', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="## Centrale Marseille – Présentation générale  \n\n**Centrale Marseille** (souvent appelée *École Centrale de Marseille* ou simplement *Centrale Marseille*) est l’une des écoles d’ingénieurs françaises les plus reconnues, membre du réseau **Centrale** (qui comprend également CentraleSupélec, Centrale Lille, Centrale Lyon, Centrale Nantes et Centrale Paris). Elle fait partie du **Groupement des écoles Centrale (GEC)** et s’inscrit dans la tradition d’excellence académique, d’innovation technologique et d’ouverture internationale.\n\n| Élément | Détails |\n|---------|---------|\n| **Création** | 1891, sous le nom de *École Supérieure d’Ingénieurs de Marseille* (ESIM). Rebrandée en *Centrale Marseille* en 2009 lorsqu’elle a intégré le réseau Centralien. |\n| **Statut** | École publique d’ingénieurs, rattachée à l’Université d’Aix‑Mar

In [13]:
# print(response.choices[0].message.content)

Choisir un modèle

Construire la prompt

In [22]:
df["abstract"].iloc[9]

'The past 15 years have witnessed a remarkable increase in both the scale and scope of social and behavioral data available to researchers, leading some to herald the emergence of a new field: "computational social science." Against these exciting developments stands a stubborn fact: that in spite of many thousands of published papers, there has been surprisingly little progress on the "big" questions that motivated the field in the first place?questions concerning systemic risk in financial systems, problem solving in complex organizations, and the dynamics of epidemics or social movements, among others. In this talk I highlight some examples of research that would not have been possible just a handful of years ago and that illustrate the promise of CSS. At the same time, they illustrate its limitations. I then conclude with some thoughts on how CSS can bridge the gap between its current state and its potential.'

In [23]:
messages = [
  {
    "role": "system",
    "content": "You are an efficient research assistant helping with text annotation."
  },
  {
    "role": "user",
    "content": """
    Annotate this following text if its topic is about AI or algorithms.
    If so, returns AI, and otherwise, return NOT AI.
    Don't return anything else.
    """
      + df["abstract"].iloc[9]
  }
]

In [24]:
#!pip install openai
from openai import OpenAI
api_url = "https://openrouter.ai/api/v1"
 
client = OpenAI(
  base_url=api_url,
  api_key=api_key,
)

response = client.chat.completions.create(
    model="openai/gpt-oss-120b",
    messages=messages
)
response.choices[0].message.content

'NOT AI'

### Application Faire une boucle sur une liste de documents : évaluer si un abstract porte ou pas sur l'IA

- Tester avec des exemples (few shot)
- Tester avec du raisonnement (chaîne of thought)

In [25]:
client = OpenAI(
  base_url=api_url,
  api_key=api_key,
)
def annotate_abstract(abstract, client):
    """
    Annotater an element
    """
    try:
        messages = [
              {
                "role": "system",
                "content": "You are an efficient research assistant helping with text annotation."
              },
              {
                "role": "user",
                "content": """
                Annotate this following text if its topic is about AI or algorithms.
                If so, returns AI, and otherwise, return NOT AI.
                Don't return anything else.
                """
                  + abstract
              }
            ]
        response = client.chat.completions.create(
        model="openai/gpt-oss-120b",
        messages=messages
        )
        return response.choices[0].message.content
    except:
        print("Error")
        return None
        
    

In [27]:
annotations = []
i=0
for abstract in df["abstract"][0:10]:
    print(f"annotate element {i}")
    annotation = annotate_abstract(abstract, client)
    annotations.append(annotation)
    i+=1

annotate element 0
annotate element 1
annotate element 2
annotate element 3
annotate element 4
annotate element 5
annotate element 6
annotate element 7
annotate element 8
annotate element 9


In [28]:
annotations

['AI',
 'NOT AI',
 'NOT AI',
 'AI',
 'NOT AI',
 'NOT AI',
 'AI',
 'NOT AI',
 'NOT AI',
 'NOT AI']

## Evaluation

Comment évaluer la qualité d'un codage ? 

- La nécessité d'un gold standard
- Le calcul de métriques `from sklearn.metrics import classification_report`

### Application : construire un jeu de données d'évaluation

- Utiliser des outils permettant l'interaction humaine
- Toujours de l'ambiguité : accord inter-annotateur (alpha de Chronbach, autre)

### Mesurer la qualité d'une évaluation

Différentes métriques

- précision
- recall
- f1

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [30]:
df_annotated = pd.read_csv("../data/data_annotated.csv")
df_annotated = df_annotated[df_annotated["labels"]!="HS"]
df_annotated.head()

,id_internal,dataset,dataset_annotation,labels,user,timestamp,comment,text,dataset_row_number
1,1115,train,train,NOT AI,root,2026-02-20 12:35:43.322496,NaN,Studies in computational social science often ...,1115
2,245,train,train,NOT AI,root,2026-02-20 12:35:51.282554,NaN,Has the emerging field of computational social...,245
4,1570,train,train,NOT AI,root,2026-02-20 12:35:59.511923,NaN,Recent surveys indicate most Americans are rel...,1570
5,866,train,train,NOT AI,root,2026-02-20 12:36:41.214757,NaN,The iLCM project pursues the development of an...,866
6,316,train,train,AI,root,2026-02-20 12:36:45.164973,NaN,Agent-Based Modelling; ABM; Artificial Intelli...,316


In [33]:
annotations = []
i=0
for abstract in df_annotated["text"]:
    print(f"annotate element {i}")
    annotation = annotate_abstract(abstract, client)
    annotations.append(annotation)
    i+=1

annotate element 0
annotate element 1
annotate element 2
annotate element 3
annotate element 4
annotate element 5
annotate element 6
annotate element 7
annotate element 8
annotate element 9
annotate element 10
annotate element 11
annotate element 12
annotate element 13
annotate element 14


In [34]:
df_annotated["predicted"] = annotations

In [36]:
from sklearn.metrics import classification_report
print(classification_report(df_annotated["labels"],df_annotated["predicted"]))

              precision    recall  f1-score   support

          AI       0.58      1.00      0.74         7
      NOT AI       1.00      0.38      0.55         8

    accuracy                           0.67        15
   macro avg       0.79      0.69      0.64        15
weighted avg       0.81      0.67      0.63        15



## Application : extraire de l'information

Sur les articles identifiés comme AI-related, extraire les termes liés pour les analyser

- Construire une prompt
- Tester sur plusieurs textes
- Gérer la structure des données